## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Kudahuvadhoo,MV,2.6708,72.8944,82.71,76,73,16.62,broken clouds,0.0,0.0
1,1,Bluff,NZ,-46.6000,168.3333,51.80,56,83,13.47,broken clouds,0.0,0.0
2,2,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,71.85,95,98,3.53,overcast clouds,0.0,0.0
3,3,Wuwei,CN,37.9281,102.6414,51.40,26,2,6.22,clear sky,0.0,0.0
4,4,Hermanus,ZA,-34.4187,19.2345,53.91,66,82,8.61,broken clouds,0.0,0.0


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 60
What is the maximun temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Kudahuvadhoo,MV,2.6708,72.8944,82.71,76,73,16.62,broken clouds,0.0,0.0
2,2,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,71.85,95,98,3.53,overcast clouds,0.0,0.0
5,5,Coahuayana,MX,18.7333,-103.6833,75.09,84,10,3.27,clear sky,0.0,0.0
6,6,Rancho Palos Verdes,US,33.7445,-118.3870,61.99,92,62,3.00,broken clouds,0.0,0.0
8,8,San Patricio,US,28.0170,-97.5169,76.84,72,0,6.60,clear sky,0.0,0.0


In [6]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

284


City_ID                284
City                   284
Country                283
Lat                    284
Lng                    284
Max Temp               284
Humidity               284
Cloudiness             284
Wind Speed             284
Current Description    284
Rain (inches)          284
Snow (inches)          284
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()


283


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Kudahuvadhoo,MV,2.6708,72.8944,82.71,76,73,16.62,broken clouds,0.0,0.0
2,2,Sao Miguel Do Araguaia,BR,-13.2750,-50.1628,71.85,95,98,3.53,overcast clouds,0.0,0.0
5,5,Coahuayana,MX,18.7333,-103.6833,75.09,84,10,3.27,clear sky,0.0,0.0
6,6,Rancho Palos Verdes,US,33.7445,-118.3870,61.99,92,62,3.00,broken clouds,0.0,0.0
8,8,San Patricio,US,28.0170,-97.5169,76.84,72,0,6.60,clear sky,0.0,0.0


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kudahuvadhoo,MV,82.71,broken clouds,2.6708,72.8944,
2,Sao Miguel Do Araguaia,BR,71.85,overcast clouds,-13.2750,-50.1628,
5,Coahuayana,MX,75.09,clear sky,18.7333,-103.6833,
6,Rancho Palos Verdes,US,61.99,broken clouds,33.7445,-118.3870,
8,San Patricio,US,76.84,clear sky,28.0170,-97.5169,
10,Katsuura,JP,75.11,broken clouds,35.1333,140.3000,
11,Kapaa,US,78.78,overcast clouds,22.0752,-159.3190,
14,Busselton,AU,66.04,broken clouds,-33.6500,115.3333,
15,Hobart,AU,78.03,broken clouds,-42.8794,147.3294,
20,Alofi,NU,80.49,few clouds,-19.0595,-169.9187,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kudahuvadhoo,MV,82.71,broken clouds,2.6708,72.8944,Niyama Private Islands Maldives
2,Sao Miguel Do Araguaia,BR,71.85,overcast clouds,-13.2750,-50.1628,Hotel Executivo Palace
5,Coahuayana,MX,75.09,clear sky,18.7333,-103.6833,El rranchito michuacan de ocampos
6,Rancho Palos Verdes,US,61.99,broken clouds,33.7445,-118.3870,Terranea Resort
8,San Patricio,US,76.84,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
10,Katsuura,JP,75.11,broken clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
11,Kapaa,US,78.78,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Busselton,AU,66.04,broken clouds,-33.6500,115.3333,Observatory Guest House
15,Hobart,AU,78.03,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
20,Alofi,NU,80.49,few clouds,-19.0595,-169.9187,Taloa Heights


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))